In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "prcptot"

In [4]:
#Variable 
variable85="rcp85_"+var+"_monthly"
variable45="rcp45_"+var+"_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_prcptot_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_prcptot_monthly.nc']

In [5]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 12 dataframes

In [7]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)         
    df45f.columns = [month + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined


KeyboardInterrupt: 

In [ ]:
df45list


In [22]:
def merge_multiple_dfs(df_list, on):
    df_result = df_list[0]
    for df in df_list[1:]:
        df_result = df_result.merge(df, on=on)
    return df_result

In [12]:
mkey = ["lat", "lon"]

In [ ]:
dfse45 = merge_multiple_dfs(df45list[0:12], on=mkey)

In [ ]:
dfse452 = dfse45.reset_index()

In [ ]:
dfse452.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [9]:
dfse452 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [10]:
dfse453 = dfse452.drop(columns=["january_hist_rcp45_p10","january_hist_rcp45_p50", "january_hist_rcp45_p90", 
                               "february_hist_rcp45_p10", "february_hist_rcp45_p50", "february_hist_rcp45_p90", 
                               "march_hist_rcp45_p10","march_hist_rcp45_p50", "march_hist_rcp45_p90", 
                               "april_hist_rcp45_p10","april_hist_rcp45_p50", "april_hist_rcp45_p90",
                              "may_hist_rcp45_p10","may_hist_rcp45_p50", "may_hist_rcp45_p90", 
                               "june_hist_rcp45_p10", "june_hist_rcp45_p50", "june_hist_rcp45_p90", 
                               "july_hist_rcp45_p10","july_hist_rcp45_p50", "july_hist_rcp45_p90", 
                               "august_hist_rcp45_p10","august_hist_rcp45_p50", "august_hist_rcp45_p90",
                              "september_hist_rcp45_p10","september_hist_rcp45_p50", "september_hist_rcp45_p90", 
                               "october_hist_rcp45_p10", "october_hist_rcp45_p50", "october_hist_rcp45_p90", 
                               "november_hist_rcp45_p10","november_hist_rcp45_p50", "november_hist_rcp45_p90", 
                               "december_hist_rcp45_p10","december_hist_rcp45_p50", "december_hist_rcp45_p90"])
dfse453


,lat,lon,january_t2050_rcp45_p10,january_t2080_rcp45_p10,january_t2050_rcp45_p50,january_t2080_rcp45_p50,january_t2050_rcp45_p90,january_t2080_rcp45_p90,february_t2050_rcp45_p10,february_t2080_rcp45_p10,...,november_t2050_rcp45_p50,november_t2080_rcp45_p50,november_t2050_rcp45_p90,november_t2080_rcp45_p90,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,23.31,23.48,44.78,44.25,71.75,79.26,25.46,28.02,...,81.08,79.88,129.02,129.71,36.77,33.30,67.30,75.41,105.42,116.96
1,40.041039,-88.961884,23.93,24.02,45.83,45.49,73.30,81.28,26.39,28.96,...,81.63,80.56,130.52,130.58,37.54,33.98,68.74,76.71,107.11,119.16
2,40.041039,-88.878548,24.50,24.66,46.54,45.97,73.58,81.40,26.48,29.06,...,81.26,79.99,128.71,128.38,38.29,34.84,69.56,77.02,106.47,118.31
3,40.041039,-88.795219,24.78,24.97,46.42,45.90,73.38,81.11,26.09,28.60,...,81.06,79.79,127.22,126.77,38.83,35.30,69.62,76.58,105.45,116.51
4,40.041039,-88.711891,25.09,25.34,47.10,46.90,75.32,82.81,25.45,27.81,...,81.61,80.25,128.14,127.52,39.46,36.02,70.99,77.60,107.77,118.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,15.27,13.51,38.49,34.54,87.94,81.96,9.04,11.72,...,53.98,53.91,99.71,95.81,20.25,21.36,46.20,49.95,93.79,92.90
76202,66.623306,-61.546310,15.92,13.98,39.18,35.43,88.78,83.03,9.53,12.45,...,55.24,55.25,101.16,97.45,21.14,22.17,47.11,51.29,94.70,94.61
76203,66.623306,-61.462978,15.56,13.60,38.83,34.98,88.44,82.73,9.19,11.99,...,54.35,54.57,99.81,96.60,20.81,21.72,46.78,50.47,94.09,94.03
76204,66.623306,-61.379650,15.47,13.42,38.48,34.76,87.62,82.20,9.01,11.81,...,53.85,54.14,99.01,95.88,20.81,21.61,46.36,50.36,93.25,93.40


In [11]:
lisdfmoh45 = []
for month in monthlist: 
    dfmoh = dfse452[["lat", "lon", month+"_hist_rcp45_p50"]]
    print (month)
    lisdfmoh45.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [ ]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

## Loop to create all 12 dataframes

In [ ]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)
    df85f.columns = [month + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

In [ ]:
df85list


In [ ]:
dfse85 = merge_multiple_dfs(df85list[0:12], on=mkey).reset_index()

In [ ]:
dfse85

In [22]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [13]:
dfse85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [14]:
dfse852 = dfse85.drop(columns=["january_hist_rcp85_p10","january_hist_rcp85_p50", "january_hist_rcp85_p90", 
                               "february_hist_rcp85_p10", "february_hist_rcp85_p50", "february_hist_rcp85_p90", 
                               "march_hist_rcp85_p10","march_hist_rcp85_p50", "march_hist_rcp85_p90", 
                               "april_hist_rcp85_p10","april_hist_rcp85_p50", "april_hist_rcp85_p90",
                              "may_hist_rcp85_p10","may_hist_rcp85_p50", "may_hist_rcp85_p90", 
                               "june_hist_rcp85_p10", "june_hist_rcp85_p50", "june_hist_rcp85_p90", 
                               "july_hist_rcp85_p10","july_hist_rcp85_p50", "july_hist_rcp85_p90", 
                               "august_hist_rcp85_p10","august_hist_rcp85_p50", "august_hist_rcp85_p90",
                              "september_hist_rcp85_p10","september_hist_rcp85_p50", "september_hist_rcp85_p90", 
                               "october_hist_rcp85_p10", "october_hist_rcp85_p50", "october_hist_rcp85_p90", 
                               "november_hist_rcp85_p10","november_hist_rcp85_p50", "november_hist_rcp85_p90", 
                               "december_hist_rcp85_p10","december_hist_rcp85_p50", "december_hist_rcp85_p90"])
dfse852


,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2050_rcp85_p50,november_t2080_rcp85_p50,november_t2050_rcp85_p90,november_t2080_rcp85_p90,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,25.03,24.41,46.30,49.71,76.82,82.15,31.35,31.10,...,80.22,82.81,129.28,135.33,37.02,39.59,71.20,76.09,121.86,134.99
1,40.041039,-88.961884,25.66,25.12,47.65,51.06,78.71,84.24,32.36,32.37,...,80.80,83.45,130.92,136.71,37.67,40.36,72.71,77.51,124.48,137.43
2,40.041039,-88.878548,26.22,25.71,48.16,51.53,79.07,84.89,32.49,32.65,...,80.25,82.96,129.31,134.65,38.37,40.82,73.36,78.07,123.66,135.79
3,40.041039,-88.795219,26.53,25.96,48.20,51.32,78.71,84.49,32.09,31.99,...,79.35,82.26,128.06,132.74,38.89,40.99,73.45,78.02,121.65,133.56
4,40.041039,-88.711891,26.98,26.40,49.12,52.04,80.33,86.50,31.20,31.37,...,79.38,82.90,128.83,133.40,39.55,41.76,75.07,79.42,122.99,135.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,15.97,20.20,38.45,47.24,85.22,99.26,10.93,13.59,...,53.30,62.33,98.92,111.75,20.86,24.57,47.97,52.73,96.55,98.93
76202,66.623306,-61.546310,16.56,21.00,39.43,48.42,86.30,100.27,11.56,14.25,...,54.54,63.73,100.45,113.04,21.72,25.56,49.15,53.73,98.03,100.39
76203,66.623306,-61.462978,16.13,20.54,38.98,47.85,85.92,99.12,11.14,13.72,...,53.43,62.80,99.37,111.97,21.49,25.07,48.78,52.94,97.51,99.22
76204,66.623306,-61.379650,15.96,20.37,38.67,47.63,85.10,98.34,10.94,13.46,...,52.97,62.41,98.64,111.02,21.28,24.87,48.66,52.55,96.94,98.30


In [15]:
lisdfmoh85 = []
for month in monthlist: 
    dfmoh = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    print (month)
    lisdfmoh85.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


In [16]:
lisdfmoh85

[             lat        lon  january_hist_rcp85_p50
 0      40.041039 -89.045212                   41.72
 1      40.041039 -88.961884                   42.96
 2      40.041039 -88.878548                   43.71
 3      40.041039 -88.795219                   43.86
 4      40.041039 -88.711891                   44.82
 ...          ...        ...                     ...
 76201  66.623306 -61.629639                   27.66
 76202  66.623306 -61.546310                   28.39
 76203  66.623306 -61.462978                   27.76
 76204  66.623306 -61.379650                   27.64
 76205  66.623306 -61.296322                   25.86
 
 [76206 rows x 3 columns],              lat        lon  february_hist_rcp85_p50
 0      40.041039 -89.045212                    50.43
 1      40.041039 -88.961884                    52.19
 2      40.041039 -88.878548                    52.26
 3      40.041039 -88.795219                    51.24
 4      40.041039 -88.711891                    50.36
 ...        

# Merge df45 and 85

# Extract hist

## Getting historic mean for each month

In [17]:
### Function
def mean_month(month, dfse45, dfse85):
    print(month)
    dfw45 = dfse45[["lat", "lon", month+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[month+"_hist_p50"] = round((dfsew[month+"_hist_rcp45_p50"]+ dfsew[month+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", month+"_hist_p50"]]
    return (dftgwh3)


### All months historic mean

In [18]:
listdfmo = []
i=0
for month in monthlist:
    print(month)
    dfm = mean_month(month, lisdfmoh45[i], lisdfmoh85[i])
    i = i+1
    listdfmo.append(dfm)

january
january
february
february
march
march
april
april
may
may
june
june
july
july
august
august
september
september
october
october
november
november
december
december


# Merge means with the 45 and 85 DF

In [19]:
dftgseall = dfse852.merge(dfse452, on=mkey)
i=0
for df in listdfmo:
    dftgseall2 = dftgseall.merge(listdfmo[i], on=mkey)
    i = i +1


In [20]:
dftgseall

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,25.03,24.41,46.30,49.71,76.82,82.15,31.35,31.10,...,129.71,37.47,36.77,33.30,66.50,67.30,75.41,110.00,105.42,116.96
1,40.041039,-88.961884,25.66,25.12,47.65,51.06,78.71,84.24,32.36,32.37,...,130.58,38.13,37.54,33.98,67.60,68.74,76.71,111.89,107.11,119.16
2,40.041039,-88.878548,26.22,25.71,48.16,51.53,79.07,84.89,32.49,32.65,...,128.38,38.95,38.29,34.84,68.08,69.56,77.02,111.08,106.47,118.31
3,40.041039,-88.795219,26.53,25.96,48.20,51.32,78.71,84.49,32.09,31.99,...,126.77,39.38,38.83,35.30,68.07,69.62,76.58,109.50,105.45,116.51
4,40.041039,-88.711891,26.98,26.40,49.12,52.04,80.33,86.50,31.20,31.37,...,127.52,39.83,39.46,36.02,69.48,70.99,77.60,110.83,107.77,118.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,15.97,20.20,38.45,47.24,85.22,99.26,10.93,13.59,...,95.81,15.64,20.25,21.36,40.05,46.20,49.95,79.69,93.79,92.90
76202,66.623306,-61.546310,16.56,21.00,39.43,48.42,86.30,100.27,11.56,14.25,...,97.45,16.40,21.14,22.17,40.86,47.11,51.29,80.72,94.70,94.61
76203,66.623306,-61.462978,16.13,20.54,38.98,47.85,85.92,99.12,11.14,13.72,...,96.60,16.16,20.81,21.72,40.12,46.78,50.47,80.06,94.09,94.03
76204,66.623306,-61.379650,15.96,20.37,38.67,47.63,85.10,98.34,10.94,13.46,...,95.88,16.15,20.81,21.61,39.99,46.36,50.36,79.56,93.25,93.40


In [23]:
dfmop50 = merge_multiple_dfs(listdfmo[0:12], on=mkey)

In [24]:
dfmop50

,lat,lon,january_hist_p50,february_hist_p50,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,42.32,50.16,78.17,97.00,99.51,96.96,103.85,94.08,75.47,68.32,76.13,66.41
1,40.041039,-88.961884,43.54,51.92,79.68,97.96,100.44,98.68,104.00,91.86,74.82,68.48,76.88,67.54
2,40.041039,-88.878548,44.22,51.98,78.95,96.74,99.34,98.57,102.42,92.02,73.92,67.78,76.70,67.94
3,40.041039,-88.795219,44.33,50.88,78.23,95.84,98.36,98.65,102.59,93.90,72.85,67.22,76.47,67.87
4,40.041039,-88.711891,45.22,50.10,77.39,95.38,96.89,99.20,105.44,95.26,71.97,66.14,76.90,69.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,28.38,23.10,22.98,28.18,37.76,27.10,31.86,43.32,51.56,62.60,46.67,39.19
76202,66.623306,-61.546310,29.11,23.86,24.02,28.91,38.45,28.16,32.62,43.96,52.24,63.72,47.61,39.99
76203,66.623306,-61.462978,28.56,23.33,23.44,28.58,38.00,27.46,32.06,43.32,51.80,63.23,46.90,39.54
76204,66.623306,-61.379650,28.34,22.88,23.12,28.49,37.64,27.27,31.81,43.00,51.33,62.64,46.39,39.33


In [25]:
dftgseall2 = dfse852.merge(dfse452, on=mkey)

In [26]:
#dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [27]:
dftgseall2

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,25.03,24.41,46.30,49.71,76.82,82.15,31.35,31.10,...,129.71,37.47,36.77,33.30,66.50,67.30,75.41,110.00,105.42,116.96
1,40.041039,-88.961884,25.66,25.12,47.65,51.06,78.71,84.24,32.36,32.37,...,130.58,38.13,37.54,33.98,67.60,68.74,76.71,111.89,107.11,119.16
2,40.041039,-88.878548,26.22,25.71,48.16,51.53,79.07,84.89,32.49,32.65,...,128.38,38.95,38.29,34.84,68.08,69.56,77.02,111.08,106.47,118.31
3,40.041039,-88.795219,26.53,25.96,48.20,51.32,78.71,84.49,32.09,31.99,...,126.77,39.38,38.83,35.30,68.07,69.62,76.58,109.50,105.45,116.51
4,40.041039,-88.711891,26.98,26.40,49.12,52.04,80.33,86.50,31.20,31.37,...,127.52,39.83,39.46,36.02,69.48,70.99,77.60,110.83,107.77,118.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,15.97,20.20,38.45,47.24,85.22,99.26,10.93,13.59,...,95.81,15.64,20.25,21.36,40.05,46.20,49.95,79.69,93.79,92.90
76202,66.623306,-61.546310,16.56,21.00,39.43,48.42,86.30,100.27,11.56,14.25,...,97.45,16.40,21.14,22.17,40.86,47.11,51.29,80.72,94.70,94.61
76203,66.623306,-61.462978,16.13,20.54,38.98,47.85,85.92,99.12,11.14,13.72,...,96.60,16.16,20.81,21.72,40.12,46.78,50.47,80.06,94.09,94.03
76204,66.623306,-61.379650,15.96,20.37,38.67,47.63,85.10,98.34,10.94,13.46,...,95.88,16.15,20.81,21.61,39.99,46.36,50.36,79.56,93.25,93.40


In [28]:
dftgseall3 = dftgseall2.merge(dfmop50, on=mkey)

In [29]:
dftgseall3.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_monthly.feather")

In [30]:
dftgseall3

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,25.03,24.41,46.30,49.71,76.82,82.15,31.35,31.10,...,78.17,97.00,99.51,96.96,103.85,94.08,75.47,68.32,76.13,66.41
1,40.041039,-88.961884,25.66,25.12,47.65,51.06,78.71,84.24,32.36,32.37,...,79.68,97.96,100.44,98.68,104.00,91.86,74.82,68.48,76.88,67.54
2,40.041039,-88.878548,26.22,25.71,48.16,51.53,79.07,84.89,32.49,32.65,...,78.95,96.74,99.34,98.57,102.42,92.02,73.92,67.78,76.70,67.94
3,40.041039,-88.795219,26.53,25.96,48.20,51.32,78.71,84.49,32.09,31.99,...,78.23,95.84,98.36,98.65,102.59,93.90,72.85,67.22,76.47,67.87
4,40.041039,-88.711891,26.98,26.40,49.12,52.04,80.33,86.50,31.20,31.37,...,77.39,95.38,96.89,99.20,105.44,95.26,71.97,66.14,76.90,69.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,15.97,20.20,38.45,47.24,85.22,99.26,10.93,13.59,...,22.98,28.18,37.76,27.10,31.86,43.32,51.56,62.60,46.67,39.19
76202,66.623306,-61.546310,16.56,21.00,39.43,48.42,86.30,100.27,11.56,14.25,...,24.02,28.91,38.45,28.16,32.62,43.96,52.24,63.72,47.61,39.99
76203,66.623306,-61.462978,16.13,20.54,38.98,47.85,85.92,99.12,11.14,13.72,...,23.44,28.58,38.00,27.46,32.06,43.32,51.80,63.23,46.90,39.54
76204,66.623306,-61.379650,15.96,20.37,38.67,47.63,85.10,98.34,10.94,13.46,...,23.12,28.49,37.64,27.27,31.81,43.00,51.33,62.64,46.39,39.33


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [31]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [32]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [33]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [35]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    dfpolyshape = 0
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    dftp = 0
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")
    gdf = 0

DDE_STF_20K_REG_FOR_VUE_S


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Estrie-Monteregie-Laval-Montreal_prcptot_monthly.json: No such file or directory


Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Outaouais_prcptot_monthly.json: No such file or directory


OUTAOUAIS


...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_prcptot_monthly.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
